#### **Desarrollo de la API con FastAPI**

En este notebook, desarrollaré una API utilizando FastAPI para exponer los datos de la empresa. 

Implementaré seis funciones como endpoints, cada una decorada con @app.get('/').

- cantidad_filmaciones_mes(Mes): Devuelve la cantidad de películas estrenadas en el mes consultado.
- cantidad_filmaciones_dia(Dia): Retorna la cantidad de películas estrenadas en el día indicado.
- score_titulo(titulo_de_la_filmación): Proporciona el título, año de estreno y score de la película.
- votos_titulo(titulo_de_la_filmación): Muestra la cantidad de votos y promedio de votaciones, verificando que haya al menos 2000 valoraciones.
- get_actor(nombre_actor): Devuelve el éxito de un actor, incluyendo la cantidad de películas y el promedio de retorno.
- get_director(nombre_director): Muestra el éxito de un director, con detalles de las películas que ha dirigido.

Con estas funciones, facilitaré el acceso a información clave de la empresa.

#### **Importo las Librerías Necesarias**

In [24]:
import pandas as pd
import ast
import json
import numpy as np 
import pyarrow.parquet as pq   


#### **Cargo los archivos Parquet**

In [25]:
# Definir las rutas de los archivos
ruta_movies = 'Datasets/movies_df.parquet'
ruta_credits = 'Datasets/credits_df.parquet'
ruta_crew = 'Datasets/crew_df.parquet'
ruta_cast = 'Datasets/cast_df.parquet'

# Cargar los archivos Parquet en DataFrames
movies_df = pd.read_parquet(ruta_movies)
credits_df = pd.read_parquet(ruta_credits)
crew_df = pd.read_parquet(ruta_crew)
cast_df = pd.read_parquet(ruta_cast)

In [26]:
movies_df

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,title,vote_average,vote_count,production_company_names,genre_names,country_names,language_names,release_year,return
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,Toy Story,7.7,5415.0,Pixar Animation Studios,"Animation, Comedy, Family",United States of America,English,1995,12.451801
1,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Jumanji,6.9,2413.0,"TriStar Pictures, Teitler Film, Interscope Com...","Adventure, Fantasy, Family",United States of America,"English, Français",1995,4.043035
2,0,15602,en,A family wedding reignites the ancient feud be...,11.712900,1995-12-22,0.0,101.0,Released,Grumpier Old Men,6.5,92.0,"Warner Bros., Lancaster Gate","Romance, Comedy",United States of America,English,1995,0.000000
3,16000000,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,Waiting to Exhale,6.1,34.0,Twentieth Century Fox Film Corporation,"Comedy, Drama, Romance",United States of America,English,1995,5.090760
4,0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,Father of the Bride Part II,5.7,173.0,"Sandollar Productions, Touchstone Pictures",Comedy,United States of America,English,1995,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44339,0,30840,en,"Yet another version of the classic epic, with ...",5.683753,1991-05-13,0.0,104.0,Released,Robin Hood,5.7,26.0,"Westdeutscher Rundfunk (WDR), Working Title Fi...","Drama, Action, Romance","Canada, Germany, United Kingdom, United States...",English,1991,0.000000
44340,0,111109,tl,An artist struggles to finish his work while a...,0.178241,2011-11-17,0.0,360.0,Released,Century of Birthing,9.0,3.0,Sine Olivia,Drama,Philippines,,2011,0.000000
44341,0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0.0,90.0,Released,Betrayal,3.8,6.0,American World Pictures,"Action, Drama, Thriller",United States of America,English,2003,0.000000
44342,0,227506,en,"In a small town live two brothers, one a minis...",0.003503,1917-10-21,0.0,87.0,Released,Satan Triumphant,0.0,0.0,Yermoliev,,Russia,,1917,0.000000


In [27]:
cast_df

,id,cast_name,character,gender
0,862,Tom Hanks,Woody (voice),2
1,862,Tim Allen,Buzz Lightyear (voice),2
2,862,Don Rickles,Mr. Potato Head (voice),2
3,862,Jim Varney,Slinky Dog (voice),2
4,862,Wallace Shawn,Rex (voice),2
...,...,...,...,...
495090,67758,Joe Sabatino,Sammy Benetto,2
495091,67758,Kiko Ellsworth,Steve,2
495092,67758,Don Swayze,Fred,2
495093,67758,Peter Dobson,"Artie, Hitman #1",2


In [28]:
crew_df

,id,crew_name,crew_job,department,gender
0,862,John Lasseter,Director,Directing,2
1,862,Joss Whedon,Screenplay,Writing,2
2,862,Andrew Stanton,Screenplay,Writing,2
3,862,Joel Cohen,Screenplay,Writing,2
4,862,Alec Sokolow,Screenplay,Writing,0
...,...,...,...,...,...
464309,67758,Richard McHugh,Original Music Composer,Sound,0
464310,67758,João Fernandes,Director of Photography,Camera,2
464311,227506,Yakov Protazanov,Director,Directing,0
464312,227506,Joseph N. Ermolieff,Producer,Production,2


#### **Cantidad de Filmaciones por Mes**

In [29]:
# Crear la función para contar filmaciones por mes
def cantidad_filmaciones_mes(mes):
    # Diccionario de meses para convertir nombres en números de mes
    meses = {
        'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4,
        'mayo': 5, 'junio': 6, 'julio': 7, 'agosto': 8,
        'septiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12
    }
    
    # Convertir el nombre del mes a número
    mes_numero = meses.get(mes.lower())
    
    if mes_numero is None:
        return "Mes no válido. Por favor, ingresa un mes en español."
    
    # Asegurarse de que la columna 'release_date' sea de tipo datetime
    if not pd.api.types.is_datetime64_any_dtype(movies_df['release_date']):
        movies_df['release_date'] = pd.to_datetime(movies_df['release_date'], errors='coerce')
    
    # Filtrar las filas que coincidan con el mes y que tengan una fecha válida
    cantidad = movies_df[movies_df['release_date'].dt.month == mes_numero].shape[0]
    return f"{cantidad} películas fueron estrenadas en el mes de {mes.capitalize()}"



In [30]:
print(cantidad_filmaciones_mes('junio'))  # Debería retornar algo como "X cantidad de películas fueron estrenadas en el mes de Enero"

3101 películas fueron estrenadas en el mes de Junio


#### **Cantidad de Filmaciones por Día**

In [31]:
# Asegúrate de que 'release_date' esté en formato de fecha
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'], errors='coerce')

# Crear la función para contar filmaciones por día
def cantidad_filmaciones_dia(dia):
    # Diccionario de días para convertir nombres en números de día
    dias = {
        'lunes': 0, 'martes': 1, 'miercoles': 2, 'jueves': 3,
        'viernes': 4, 'sabado': 5, 'domingo': 6
    }
    
    # Convertir el nombre del día a número
    dia_numero = dias.get(dia.lower())
    
    if dia_numero is not None:
        # Filtrar las filas que coincidan con el día y que tengan una fecha válida
        cantidad = movies_df[movies_df['release_date'].dt.dayofweek == dia_numero].shape[0]
        return f"{cantidad} películas fueron estrenadas en los días {dia.capitalize()}"
    else:
        return "Día no válido. Por favor, ingresa un día de la semana en español."


In [32]:
print(cantidad_filmaciones_dia('jueves'))  # Debería retornar algo como "X cantidad de películas fueron estrenadas en los días Lunes"

7303 películas fueron estrenadas en los días Jueves


#### **Score de Título**

In [33]:
# Asegúrate de que 'release_date' esté en formato de fecha
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'], errors='coerce')

# Crear la función para obtener el score de una película por su título
def score_titulo(titulo):
    # Filtrar la película por el título
    pelicula = movies_df[movies_df['title'].str.lower() == titulo.lower()]
    
    if not pelicula.empty:
        # Obtener el primer resultado
        pelicula_info = pelicula.iloc[0]
        
        # Extraer el título, año de estreno y score de popularidad
        titulo = pelicula_info['title']
        anio_estreno = pelicula_info['release_date'].year
        score_popularidad = pelicula_info['popularity']
        
        return f"Título: {titulo}, Año de Estreno: {anio_estreno}, Score de Popularidad: {score_popularidad}"
    else:
        return "Película no encontrada."



In [34]:
print(score_titulo('tarzan'))  # Debería retornar el título, el año de estreno y el score de popularidad.

Título: Tarzan, Año de Estreno: 1999, Score de Popularidad: 12.453452


#### **Votos de Título**

In [35]:
def votos_titulo(titulo_de_la_filmación):
    # Buscar la película por título, asegurando que no haya valores nulos
    pelicula = movies_df[movies_df['title'].fillna("").str.contains(titulo_de_la_filmación, case=False)]
    
    if not pelicula.empty:
        cantidad_votos = pelicula['vote_count'].values[0]
        valor_promedio = pelicula['vote_average'].values[0]
        
        # Comprobar si cumple la condición de votos
        if cantidad_votos > 0:
            return f"Título: {titulo_de_la_filmación}, Cantidad de Votos: {cantidad_votos}, Valor Promedio: {valor_promedio}"
        else:
            return f"La película '{titulo_de_la_filmación}' no tiene votos registrados."
    else:
        return "Película no encontrada."


In [36]:
print(votos_titulo('outlander'))  # Debería retornar el título, cantidad de votos y valor promedio, o indicar que no cumple la condición de votos.


Título: outlander, Cantidad de Votos: 354.0, Valor Promedio: 6.1


#### **Obtener Datos de Actor**

In [37]:
# Asegúrate de que ambos DataFrames tengan la columna 'id' como tipo int64
cast_df['id'] = cast_df['id'].astype(int)
movies_df['id'] = movies_df['id'].astype(int)

def get_actor(nombre_actor):
    # Normalizar el nombre del actor
    nombre_actor = nombre_actor.strip().lower()
    
    # Filtrar el DataFrame de cast para obtener solo los registros del actor especificado
    actor_movies = cast_df[cast_df['cast_name'].str.strip().str.lower() == nombre_actor]
    
    # Verificar si el actor tiene registros
    if actor_movies.empty:
        return f"El actor {nombre_actor.title()} no se encuentra en el dataset."
    
    # Eliminar duplicados basados en 'id' para contar correctamente las películas
    actor_movies = actor_movies.drop_duplicates(subset=['id'])
    
    # Obtener la cantidad de películas en las que ha participado
    cantidad_peliculas = actor_movies.shape[0]
    
    # Unir con movies_df para obtener el retorno total
    peliculas_con_revenue = actor_movies.merge(movies_df, on='id', how='left')
    
    # Manejar valores faltantes en 'revenue' reemplazándolos por 0
    peliculas_con_revenue['revenue'] = peliculas_con_revenue['revenue'].fillna(0)
    
    # Calcular el retorno total
    retorno_total = peliculas_con_revenue['revenue'].sum()
    
    # Calcular el promedio de retorno por película
    promedio_retorno = retorno_total / cantidad_peliculas if cantidad_peliculas > 0 else 0
    
    # Formatear el resultado
    return (f"El actor {nombre_actor.title()} ha participado de {cantidad_peliculas} filmaciones, "
            f"ha conseguido un retorno total de {retorno_total:.2f} con un promedio de retorno "
            f"por filmación de {promedio_retorno:.2f}.")


In [38]:
print(get_actor('Tom Cruise'))  # Debería retornar el éxito del actor, cantidad de películas y el promedio de retorno.

El actor Tom Cruise ha participado de 47 filmaciones, ha conseguido un retorno total de 10037226026.00 con un promedio de retorno por filmación de 213558000.55.


#### **Obtener Datos de Director**

In [39]:
def get_director(nombre_director):
    # Filtrar crew_df para obtener solo las filas del director especificado
    director_movies = crew_df[
        (crew_df['crew_name'].str.strip().str.lower() == nombre_director.lower()) &
        (crew_df['crew_job'].str.strip().str.lower() == 'director')
    ]

    # Verificar si el director tiene registros
    if director_movies.empty:
        return f"No se encontraron registros para el director: {nombre_director}"

    # Obtener el ID de las películas que dirigió el director
    movie_ids = director_movies['id'].unique()

    # Filtrar las películas de movies_df basadas en los IDs de las películas en crew_df
    director_movies_details = movies_df[movies_df['id'].isin(movie_ids)]

    # Calcular el retorno total y el número de películas
    total_return = director_movies_details['return'].sum()
    number_of_movies = director_movies_details.shape[0]
    average_return_per_movie = total_return / number_of_movies if number_of_movies > 0 else 0

    # Formatear el retorno para que se vea más limpio
    total_return_rounded = round(total_return, 2)
    average_return_rounded = round(average_return_per_movie, 2)

    # Mensaje de salida personalizado
    return f"El director {nombre_director} ha conseguido un retorno total de {total_return_rounded:.2f} con un promedio de {average_return_rounded:.2f} por filmación."


In [40]:
print(get_director('Steven Spielberg'))  # Debería devolver el éxito del director y el detalle de cada película relevante.


El director Steven Spielberg ha conseguido un retorno total de 310.00 con un promedio de 9.39 por filmación.


#### **Estructura de la función**

La función debe recibir un nombre de actor y devolver la cantidad de películas en las que ha participado, el retorno total logrado y el promedio de retorno por película.

In [48]:
def recomendacion(titulo, movies_df):
    # Obtener las columnas reales y comparar
    available_columns = movies_df.columns.tolist()
    
    # Definir las columnas de géneros que esperas encontrar
    expected_genres = ['Action', 'Adventure', 'Animation', 'Comedy']
    genre_columns = [genre for genre in expected_genres if genre in available_columns]
    
    # Validar que haya columnas de géneros en el DataFrame
    if not genre_columns:
        return "Error: No se encontraron columnas de géneros en el DataFrame."
    
    # Seleccionar solo las columnas de géneros presentes
    movies_df_genres = movies_df[genre_columns]
    
    # Verificar que la película existe en el DataFrame
    if titulo not in movies_df['title'].values:
        return "Película no encontrada"
    
    # Obtener el índice de la película en el DataFrame
    idx = movies_df.index[movies_df['title'] == titulo][0]
    
    # Calcular la similitud coseno entre las películas basadas en los géneros
    similitud = cosine_similarity([movies_df_genres.iloc[idx]], movies_df_genres)[0]
    
    # Seleccionar los índices de las 5 películas más similares, excluyendo la película de entrada
    similares = sorted(list(enumerate(similitud)), key=lambda x: x[1], reverse=True)[1:6]
    recomendaciones = [movies_df.iloc[i[0]]['title'] for i in similares]
    
    return {'lista recomendada': recomendaciones}



In [47]:
# Llamar a la función y ver el resultado
recomendaciones = recomendacion("tarzan", movies_df)
print(recomendaciones)

Error: No se encontraron columnas de géneros en el DataFrame.
